### Colab only
Run the following commands for Colab and skip this section if you are using Workbench.

In [ ]:
import sys

if "google.colab" in sys.modules:
    ! pip3 install --upgrade google-cloud-aiplatform
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

    # Restart the notebook kernel after installs.
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

Fill following variables for experiments environment:

In [ ]:
# Cloud project id.
PROJECT_ID = "lloyds-genai24lon-2711"  # @param {type:"string"}

# The region you want to launch jobs in.
REGION = "us-central1"  # @param {type:"string"}

# The Cloud Storage bucket for storing experiments output.
BUCKET_URI = "gs://model-us-bucket"  # @param {type:"string"}

! gcloud config set project $PROJECT_ID

# The service account looks like:
# '@.iam.gserviceaccount.com'
# Please go to https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console
# and create service account with `Vertex AI User` and `Storage Object Admin` roles.
# The service account for deploying fine tuned model.
SERVICE_ACCOUNT = "502467212778-compute@developer.gserviceaccount.com"  # @param {type:"string"}

# The serving port.
SERVE_PORT = 7080

Updated property [core/project].


### Initialize Vertex AI API

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

### Define constants

In [ ]:
# The pre-built vllm serving docker image.
VLLM_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-vllm-serve:20231127_0916_RC00"

Once deployment succeeds, you can send text prompts to the endpoint. microsoft_wizardcoder-python-7b-v1_0-mg-one-click-deploy

In [ ]:
# Load an existing endpoint as below.

endpoint_name = 1188591860835155968
aip_endpoint_name = (
    f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint_name}"
)
endpoint = aiplatform.Endpoint(aip_endpoint_name)

In [ ]:

prompt = "Can you explain the following code snippet with a focus on clarity and simplicity, considering that I'm already familiar with NumPy, Pandas, Scikit-learn, Flask, Pytest, Requests, and Beautiful Soup. I want the explanations to be in three different sytles: Conceptual Overview, Real-World Analogy, Step by step breakdown. Here's my code: x = make_df('AB', [0, 1]) y = make_df('AB', [2, 3]) y.index = x.index  # make duplicate indices! display('x', 'y', 'pd.concat([x, y])')"  # @param {type:"string"}
instance = {
    "prompt": f"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{prompt}\n\n### Response:",
    "temperature": 0.1,
    "top_p": 1.0,
    "max_tokens": 512,
}
response = endpoint.predict(instances=[instance])

print(response.predictions)

["Prompt:\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCan you explain the following code snippet with a focus on clarity and simplicity, considering that I'm already familiar with NumPy, Pandas, Scikit-learn, Flask, Pytest, Requests, and Beautiful Soup. I want the explanations to be in three different sytles: Conceptual Overview, Real-World Analogy, Step by step breakdown. Here's my code: x = make_df('AB', [0, 1]) y = make_df('AB', [2, 3]) y.index = x.index  # make duplicate indices! display('x', 'y', 'pd.concat([x, y])')\n\n### Response:\nOutput:\nConceptual Overview:\r\n\r\nThe code creates two dataframes with the same index values and concatenates them together using the pd.concat() function from Pandas. The resulting dataframe will have the same index values as the original dataframes, but the values will be NaN for the second dataframe. The index values are duplicated using the line y.index = x.ind

In [ ]:
hidden_instructions = "#hidden_instruction: explain the following code snippet with a focus on clarity and simplicity, considering that I'm already familiar with NumPy, Pandas, Scikit-learn, Flask, Pytest, Requests, and Beautiful Soup. I want the explanations to be in three different sytles: Conceptual Overview, Real-World Analogy, Step by step breakdown. "

prompt = "Can you explain the following code: x = make_df('AB', [0, 1]) y = make_df('AB', [2, 3]) y.index = x.index  # make duplicate indices! display('x', 'y', 'pd.concat([x, y])')"  # @param {type:"string"}

instance = {
    "prompt": f"{hidden_instructions}### Reflection and Mimicry:\nReflect on the user's input after providing explanations. Why do you think the user preferred a specific style of explanation? Consider the context they provided about their familiarity with libraries. In your subsequent response, try to mimic the user's preferred style or address their specific context.\n\n{hidden_instructions}\n### Response:",
    "temperature": 0.1,
    "top_p": 1.0,
    "max_tokens": 512,
}
response_hidden = endpoint.predict(instances=[instance])

print(response_hidden.predictions)

['Prompt:\n### Instruction:\n#hidden_instruction: explain the following code snippet with a focus on clarity and simplicity, considering that I\'m already familiar with NumPy, Pandas, Scikit-learn, Flask, Pytest, Requests, and Beautiful Soup. I want the explanations to be in three different sytles: Conceptual Overview, Real-World Analogy, Step by step breakdown. \n\n### Reflection and Mimicry:\nReflect on the user\'s input after providing explanations. Why do you think the user preferred a specific style of explanation? Consider the context they provided about their familiarity with libraries. In your subsequent response, try to mimic the user\'s preferred style or address their specific context.\n\n#hidden_instruction: explain the following code snippet with a focus on clarity and simplicity, considering that I\'m already familiar with NumPy, Pandas, Scikit-learn, Flask, Pytest, Requests, and Beautiful Soup. I want the explanations to be in three different sytles: Conceptual Overview,

# Zero short with Hidden Instructions

In [ ]:
hidden_instructions_level1 = "#hidden_instruction_level1: Consider the user's familiarity with libraries and their provided scenarios."

hidden_instructions_level2 = "#hidden_instruction_level2: Provide three different styles of explanation - Conceptual Overview, Real-World Analogy, and Step-by-step Breakdown."

hidden_instructions_level3 = "#hidden_instruction_level3: After providing explanations, reflect on the user's input. Why do you think the user preferred a specific style of explanation? Consider the context they provided about their familiarity with libraries. In your subsequent response, try to mimic the user's preferred style or address their specific context."

prompt = "optb = OptimalBinning(name=variable,dtype='numerical',solver='cp')"  # @param {type:"string"}

instance = {
    "prompt": f"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction Level 1:\n{prompt}\n\n{hidden_instructions_level1}\n\n### Instruction Level 2:\n{hidden_instructions_level2}\n\n### Instruction Level 3:\n{hidden_instructions_level3}\n\n### Response:",
    "temperature": 0.1,
    "top_p": 1.0,
    "max_tokens": 1024,
}

response_hidden = endpoint.predict(instances=[instance])

In [ ]:
for prediction in response_hidden.predictions:
    print(prediction)

Prompt:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction Level 1:
optb = OptimalBinning(name=variable,dtype='numerical',solver='cp')

#hidden_instruction_level1: Consider the user's familiarity with libraries and their provided scenarios.

### Instruction Level 2:
#hidden_instruction_level2: Provide three different styles of explanation - Conceptual Overview, Real-World Analogy, and Step-by-step Breakdown.

### Instruction Level 3:
#hidden_instruction_level3: After providing explanations, reflect on the user's input. Why do you think the user preferred a specific style of explanation? Consider the context they provided about their familiarity with libraries. In your subsequent response, try to mimic the user's preferred style or address their specific context.

### Response:
Output:
Conceptual Overview:
Optimal Binning is a technique used to find the optimal number of bins for a numerical variable in order to min

# few shots with hidden instructions

In [ ]:
examples = "code: \"\"\", \"\"\".join('\"' + item + '\"' for item in parameters), comment: add quotes to every list element in parameters parameters. \n code: class MathOperations:\n def __init__(self, num1, num2):\n self.num1 = num1 \n self.num2 = num2 \n def add(self): \n return self.num1 + self.num2 \n def multiply(self): \n return self.num1 * self.num2, comment: This is a class to perform basic mathematical operations.The class demonstrates the usage of docstrings in a python class. __init__ is a constructor to initialize the attributes of the class. \n Args: \n num1 (int): accepts an integer value. \n num2 (int): accepts an integer value.\n The add function calculates sum of the numbers. \n Returns: \n int: Sum of num1 and num2. \n The multiply function calculates product of the numbers. \n Returns: \n int: Product of num1 and num2."
hidden_instructions_level1 = "#hidden_instruction_level1: Consider the user's familiarity with libraries and their provided scenarios."

hidden_instructions_level2 = "#hidden_instruction_level2: Provide four different styles of explanation - Conceptual Overview, Real-World Analogy, and Step-by-step Breakdown."

hidden_instructions_level3 = "#hidden_instruction_level3: After providing explanations, reflect on the user's input. Why do you think the user preferred a specific style of explanation? Consider the context they provided about their familiarity with libraries. In your subsequent response, try to mimic the user's preferred style or address their specific context."

prompt = "optb = OptimalBinning(name=variable,dtype='numerical',solver='cp')"  # @param {type:"string"}

instance = {
    "prompt": f"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction Level 1:\n{prompt}\n\n{hidden_instructions_level1}\n\n### Instruction Level 2:\n{hidden_instructions_level2}\n\n### Instruction Level 3:\n{hidden_instructions_level3}\n\n### Examples:\n{examples}\n\n### Response:",
    "temperature": 0.1,
    "top_p": 1.0,
    "max_tokens": 5000,
}

response_hidden = endpoint.predict(instances=[instance])

In [ ]:
for prediction in response_hidden.predictions:
    print(prediction)

Prompt:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction Level 1:
optb = OptimalBinning(name=variable,dtype='numerical',solver='cp')

#hidden_instruction_level1: Consider the user's familiarity with libraries and their provided scenarios.

### Instruction Level 2:
#hidden_instruction_level2: Provide four different styles of explanation - Conceptual Overview, Real-World Analogy, and Step-by-step Breakdown.

### Instruction Level 3:
#hidden_instruction_level3: After providing explanations, reflect on the user's input. Why do you think the user preferred a specific style of explanation? Consider the context they provided about their familiarity with libraries. In your subsequent response, try to mimic the user's preferred style or address their specific context.

### Examples:
code: """, """.join('"' + item + '"' for item in parameters), comment: add quotes to every list element in parameters parameters. 
 code: cla

# for sas users

In [ ]:
examples = "code: \"\"\", \"\"\".join('\"' + item + '\"' for item in parameters), comment: add quotes to every list element in parameters parameters. \n code: class MathOperations:\n def __init__(self, num1, num2):\n self.num1 = num1 \n self.num2 = num2 \n def add(self): \n return self.num1 + self.num2 \n def multiply(self): \n return self.num1 * self.num2, comment: This is a class to perform basic mathematical operations.The class demonstrates the usage of docstrings in a python class. __init__ is a constructor to initialize the attributes of the class. \n Args: \n num1 (int): accepts an integer value. \n num2 (int): accepts an integer value.\n The add function calculates sum of the numbers. \n Returns: \n int: Sum of num1 and num2. \n The multiply function calculates product of the numbers. \n Returns: \n int: Product of num1 and num2."
hidden_instructions_level1 = "#hidden_instruction_level1: Consider the user is currently a SAS user rather than python."

hidden_instructions_level2 = "#hidden_instruction_level2: Provide three different styles of explanation - Conceptual Overview, Real-World Analogy, Step-by-step Breakdown, SAS equivalent."

hidden_instructions_level3 = "#hidden_instruction_level3: After providing explanations, reflect on the user's input. Why do you think the user preferred a specific style of explanation? Consider the context they provided about their familiarity with libraries. In your subsequent response, try to mimic the user's preferred style or address their specific context."

prompt = "optb = OptimalBinning(name=variable,dtype='numerical',solver='cp')"  # @param {type:"string"}

instance = {
    "prompt": f"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction Level 1:\n{prompt}\n\n{hidden_instructions_level1}\n\n### Instruction Level 2:\n{hidden_instructions_level2}\n\n### Instruction Level 3:\n{hidden_instructions_level3}\n\n### Examples:\n{examples}\n\n### Response:",
    "temperature": 0.1,
    "top_p": 1.0,
    "max_tokens": 5000,
}

response_sas = endpoint.predict(instances=[instance])

In [ ]:
for prediction in response_sas.predictions:
    print(prediction)

Prompt:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction Level 1:
optb = OptimalBinning(name=variable,dtype='numerical',solver='cp')

#hidden_instruction_level1: Consider the user is currently a SAS user rather than python.

### Instruction Level 2:
#hidden_instruction_level2: Provide three different styles of explanation - Conceptual Overview, Real-World Analogy, Step-by-step Breakdown, SAS equivalent.

### Instruction Level 3:
#hidden_instruction_level3: After providing explanations, reflect on the user's input. Why do you think the user preferred a specific style of explanation? Consider the context they provided about their familiarity with libraries. In your subsequent response, try to mimic the user's preferred style or address their specific context.

### Examples:
code: """, """.join('"' + item + '"' for item in parameters), comment: add quotes to every list element in parameters parameters. 
 code: class

# **DEMO**

In [ ]:
examples = "code: \"\"\", \"\"\".join('\"' + item + '\"' for item in parameters), comment: add quotes to every list element in parameters parameters. \n code: class MathOperations:\n def __init__(self, num1, num2):\n self.num1 = num1 \n self.num2 = num2 \n def add(self): \n return self.num1 + self.num2 \n def multiply(self): \n return self.num1 * self.num2, comment: This is a class to perform basic mathematical operations.The class demonstrates the usage of docstrings in a python class. __init__ is a constructor to initialize the attributes of the class. \n Args: \n num1 (int): accepts an integer value. \n num2 (int): accepts an integer value.\n The add function calculates sum of the numbers. \n Returns: \n int: Sum of num1 and num2. \n The multiply function calculates product of the numbers. \n Returns: \n int: Product of num1 and num2."
hidden_instructions_level1 = "#hidden_instruction_level1: Consider the user is currently a SAS user rather than python."

hidden_instructions_level2 = "#hidden_instruction_level2: Provide three different styles of explanation - Conceptual Overview, Real-World Analogy, Step-by-step Breakdown, SAS equivalent."

prompt = "optb = OptimalBinning(name=variable,dtype='numerical',solver='cp')"  # @param {type:"string"}

instance = {
    "prompt": f"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction Level 1:\n{prompt}\n\n{hidden_instructions_level1}\n\n### Instruction Level 2:\n{hidden_instructions_level2}\n\n### Examples:\n{examples}\n\n### Response:",
    "temperature": 0.1,
    "top_p": 1.0,
    "max_tokens": 5000,
}

response_sas_2levels = endpoint.predict(instances=[instance])

In [ ]:
for prediction in response_sas_2levels.predictions:
    print(prediction)

Prompt:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction Level 1:
optb = OptimalBinning(name=variable,dtype='numerical',solver='cp')

#hidden_instruction_level1: Consider the user is currently a SAS user rather than python.

### Instruction Level 2:
#hidden_instruction_level2: Provide three different styles of explanation - Conceptual Overview, Real-World Analogy, Step-by-step Breakdown, SAS equivalent.

### Examples:
code: """, """.join('"' + item + '"' for item in parameters), comment: add quotes to every list element in parameters parameters. 
 code: class MathOperations:
 def __init__(self, num1, num2):
 self.num1 = num1 
 self.num2 = num2 
 def add(self): 
 return self.num1 + self.num2 
 def multiply(self): 
 return self.num1 * self.num2, comment: This is a class to perform basic mathematical operations.The class demonstrates the usage of docstrings in a python class. __init__ is a constructor to initialize 

# **JUDGE DEMO**

# Base case

In [ ]:
prompt = "optb = OptimalBinning(name=variable,dtype='numerical',solver='cp')"  # @param {type:"string"}

instance = {
    "prompt": f"Below is an instruction that describes a task.\n\n### Instruction:\n Explain this piece of code. {prompt}\n\n### Response:",
    "temperature": 0.1,
    "top_p": 1.0,
    "max_tokens": 200,
}

response_base = endpoint.predict(instances=[instance])

In [ ]:
for prediction in response_base.predictions:
    print(prediction)

Prompt:
Below is an instruction that describes a task.

### Instruction:
 Explain this piece of code. optb = OptimalBinning(name=variable,dtype='numerical',solver='cp')

### Response:
Output:
This code creates an instance of the OptimalBinning class from the OptimalBinning library with the specified parameters. The name parameter specifies the name of the variable that needs to be binned, dtype parameter specifies the data type of the variable as numerical, and solver parameter specifies the solver to be used as 'cp' which stands for 'CPLEX'. 


# few shot with hidden instructions for SAS users

In [ ]:
examples = "code: \"\"\", \"\"\".join('\"' + item + '\"' for item in parameters), comment: add quotes to every list element in parameters parameters. \n code: class MathOperations:\n def __init__(self, num1, num2):\n self.num1 = num1 \n self.num2 = num2 \n def add(self): \n return self.num1 + self.num2 \n def multiply(self): \n return self.num1 * self.num2, comment: This is a class to perform basic mathematical operations.The class demonstrates the usage of docstrings in a python class. __init__ is a constructor to initialize the attributes of the class. \n Args: \n num1 (int): accepts an integer value. \n num2 (int): accepts an integer value.\n The add function calculates sum of the numbers. \n Returns: \n int: Sum of num1 and num2. \n The multiply function calculates product of the numbers. \n Returns: \n int: Product of num1 and num2."
hidden_instructions_level1 = "#hidden_instruction_level1: Consider the user is currently a SAS user rather than python."

hidden_instructions_level2 = "#hidden_instruction_level2: Provide three different styles of explanation - Conceptual Overview, Real-World Analogy, Step-by-step Breakdown, SAS equivalent."

prompt = "optb = OptimalBinning(name=variable,dtype='numerical',solver='cp')"  # @param {type:"string"}

instance = {
    "prompt": f"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction Level 1:\n{prompt}\n\n{hidden_instructions_level1}\n\n### Instruction Level 2:\n{hidden_instructions_level2}\n\n### Examples:\n{examples}\n\n### Response:",
    "temperature": 0.1,
    "top_p": 1.0,
    "max_tokens": 5000,
}

response_sas_2levels = endpoint.predict(instances=[instance])

In [ ]:
for prediction in response_sas_2levels.predictions:
    print(prediction)

# **GRADIO DEMO**

In [16]:
pip install gradio

In [17]:
import gradio as gr

In [33]:
def predict(prompt):
  examples = "code: \"\"\", \"\"\".join('\"' + item + '\"' for item in parameters), comment: add quotes to every list element in parameters parameters. \n code: class MathOperations:\n def __init__(self, num1, num2):\n self.num1 = num1 \n self.num2 = num2 \n def add(self): \n return self.num1 + self.num2 \n def multiply(self): \n return self.num1 * self.num2, comment: This is a class to perform basic mathematical operations.The class demonstrates the usage of docstrings in a python class. __init__ is a constructor to initialize the attributes of the class. \n Args: \n num1 (int): accepts an integer value. \n num2 (int): accepts an integer value.\n The add function calculates sum of the numbers. \n Returns: \n int: Sum of num1 and num2. \n The multiply function calculates product of the numbers. \n Returns: \n int: Product of num1 and num2."
  hidden_instructions_level1 = "#hidden_instruction_level1: Consider the user is currently a SAS user rather than python."

  hidden_instructions_level2 = "#hidden_instruction_level2: Provide three different styles of explanation - Conceptual Overview, Real-World Analogy, Step-by-step Breakdown, SAS equivalent."

  #prompt = "optb = OptimalBinning(name=variable,dtype='numerical',solver='cp')"

  instance = {
    "prompt": f"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction Level 1:\n{prompt}\n\n{hidden_instructions_level1}\n\n### Instruction Level 2:\n{hidden_instructions_level2}\n\n### Examples:\n{examples}\n\n### Response:",
    "temperature": 0.1,
    "top_p": 1.0,
    "max_tokens": 5000,
  }

  response_sas_2levels = endpoint.predict(instances=[instance])
  for prediction in response_sas_2levels.predictions:
    return prediction

In [34]:
intefaceModel = gr.Interface(fn=predict, inputs='text', outputs='text')

In [35]:
intefaceModel.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://eb294441232f7b1e22.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# **TRIAL**

In [30]:
def predict(prompt):
  examples = "code: \"\"\", \"\"\".join('\"' + item + '\"' for item in parameters), comment: add quotes to every list element in parameters parameters."
  hidden_instructions_level1 = "#hidden_instruction_level1: Consider the user is currently a SAS user rather than python."

  hidden_instructions_level2 = "#hidden_instruction_level2: Provide three different styles of explanation - Conceptual Overview, Real-World Analogy, Step-by-step Breakdown, SAS equivalent."

  #prompt = "optb = OptimalBinning(name=variable,dtype='numerical',solver='cp')"

  instance = {
    "prompt": f"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction Level 1:\n{prompt}\n\n{hidden_instructions_level1}\n\n### Instruction Level 2:\n{hidden_instructions_level2}\n\n### Examples:\n{examples}\n\n### Response:",
    "temperature": 0.1,
    "top_p": 1.0,
    "max_tokens": 5000,
  }

  response_sas_2levels = endpoint.predict(instances=[instance])
  for prediction in response_sas_2levels.predictions:
    return prediction

In [31]:
intefaceModel = gr.Interface(fn=predict, inputs='text', outputs='text')

In [32]:
intefaceModel.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f60f421dc9db0d0a21.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
